# Imports

based on this: https://github.com/choderalab/asapdiscovery/pull/58/files#diff-281a28c750f5e4149f8b21633b9e873cbdd9cf78f7989d93d7ab88024648d9b3L35

In [ ]:
from pathlib import Path
from asapdiscovery.data.backend.openeye import oechem, oespruce, load_openeye_pdb, load_openeye_sdfs
from asapdiscovery.modeling.modeling import get_oe_prep_opts, spruce_protein, du_to_complex
from asapdiscovery.data.schema.target import PreppedTarget
from asapdiscovery.data.schema.complex import PreppedComplex

In [ ]:
data_path = Path("/Users/alexpayne/Scientific_Projects/fragment-fold/data") 

# load structure

In [ ]:
ref = load_openeye_pdb(data_path / "ns2-ns3b-zika-fragment-screen/fragment-screen-inputs/zikv_ns2b_ns3pro_apo.pdb")

In [ ]:
success, spruce_error_message, spruced = spruce_protein(ref)

# load ligands

In [ ]:
ligs = load_openeye_sdfs(data_path / "ns2-ns3b-zika-fragment-screen/fragment-screen-inputs/output/combined_ligs.sdf")

In [ ]:
ligs[0].NumAtoms()

# make new dus

In [ ]:
opts = get_oe_prep_opts()

In [ ]:
du = oechem.OEDesignUnit()
new_dus = [oespruce.OEMakeDesignUnit(du, spruced, lig, opts) for lig in ligs[:1]]

In [ ]:
complex = du_to_complex(du)

# convert to schema

In [ ]:
from asapdiscovery.data.schema.target import Target

In [ ]:
original_target = Target.from_oemol(ref, target_name="zikv_ns2b_ns3pro")

In [ ]:
prepped_complex = PreppedComplex.from_oedu(du, target_kwargs={"target_name": "zikv_ns2b_ns3pro", "target_hash": original_target.hash}, ligand_kwargs={"ligand_name": "lig0"}, )

In [ ]:
from asapdiscovery.modeling.protein_prep import ProteinPrepper

In [ ]:
protein_prepper = ProteinPrepper()

In [ ]:
protein_prepper.cache([prepped_complex], cache_dir=data_path / "ns2-ns3b-zika-fragment-screen/prepped-foldalysis")

# test if this works for a different protein

## load

In [ ]:
target = Target.from_pdb(data_path / "ns2-ns3b-zika-fragment-screen/alphafold-flavivirus-structures/NP_722463.1.pdb", target_name="NP_722463.1")

## spruce

In [ ]:
success, spruce_error_message, spruced = spruce_protein(target.to_oemol())

## align

In [ ]:
from asapdiscovery.modeling.modeling import superpose_molecule

In [ ]:
aligned, rmsd = superpose_molecule(ref_mol=ref, mobile_mol=spruced, ref_chain="B", mobile_chain="A")

## make du

In [ ]:
du = oechem.OEDesignUnit()
oespruce.OEMakeDesignUnit(du, aligned, ligs[0], opts)

In [ ]:
prepped_target = Target.from_oemol(aligned, target_name="NP_722463.1")

In [ ]:
prepped_target.to_pdb("/Users/alexpayne/Scientific_Projects/fragment-fold/data/ns2-ns3b-zika-fragment-screen/prepped-foldalysis/NP_722463.1_apo_prepped.pdb")

In [ ]:
prepped_complex = PreppedComplex.from_oedu(du, target_kwargs={"target_name": "NP_722463.1", "target_hash": target.hash}, ligand_kwargs={"ligand_name": "lig0"}, )

In [ ]:
protein_prepper = ProteinPrepper()

In [ ]:
protein_prepper.cache([prepped_complex], cache_dir=data_path / "ns2-ns3b-zika-fragment-screen/prepped-foldalysis")

# conclusion: this works!

This makes sense since I think this is what I ended up going with before. Now we just need to add it back to the asapdiscovery repo.